In [109]:
#@title Input Parameters

Calculated_Year = "2021" #@param ["2021", "2022", "2023"] {allow-input: true}

Calculated_Month = "4" #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"] {allow-input: true}

Calculated_Option = "Max Profit" #@param ["Max Profit", "Max Coverage", "Min Cost"] {allow-input: true}

print(Calculated_Year)
print(Calculated_Month)
print(Calculated_Option)

2021
4
Max Profit


In [2]:
#mount to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#go to main folder
%cd /content/gdrive/My Drive/Classroom/KEA/Exercise 

/content/gdrive/My Drive/Classroom/KEA/Exercise


##Initial excel sheets

In [90]:
# Import `load_workbook` module from `openpyxl`
from openpyxl import load_workbook

# Load in the workbook
wb = load_workbook('KEA_CH1702008_Assignment.xlsx')

# Get a sheet by name
Client_Input_Sheet = wb['Client_Input']
CallCenter_Input_Sheet = wb['CallCenter_Input']
Equation_Rules_Sheet = wb['Equation_Rules']
Reasoning_Rules_Sheet = wb['Reasoning_Rules']
FTE_Sheet = wb['FTE']
MAX_PROFIT_Sheet = wb['MAX_PROFIT']
MAX_COVERAGE_Sheet = wb['MAX_COVERAGE']
MIN_COST_Sheet = wb['MIN_COST']
STRATEGY_Sheet = wb['STRATEGY']
STEPS_Sheet = wb['STEPS']


##Global params

In [91]:
#Strategy sheet
FINAL_TALK_TIME = []
FINAL_CONNECTED_CLIENT = []
FINAL_ATTEMPTED_CLIENT = []
FINAL_SIGNED = []
FINAL_SALES = []
FINAL_LTV = []
FINAL_TELCO_COST = []
FINAL_BONUS = []
FINAL_CALL_COVERAGE = []
FINAL_OPERATION_COST = []

#Calculate sheets
TOTAL_FTE_SC = []
FTE_Selected = 0
FTE_Selected_Remaining = 0
FINAL_CLIENT = []

#column & row header
Col_SEGMENT_CODE = 2
Col_CNT_CLIENT = 4
Col_CONNECTION_RATE = 5
Col_AVG_CALL_DURATION = 6
Col_TAKE_UP_RATE = 7
Col_AVERAGE_TICKET = 8
Col_AVG_BONUS = 9
Row_CNT_AGENT = 2
Row_LTV = 3
Row_AVG_TALKTIME = 4
Row_TELCO_COST = 5
Row_WORKING_MODE = 6
Row_SUNDAY = 7
Row_YEAR = 8
Row_MONTH = 9
Row_STRATEGY = 10

#main
run = 1
errorMsg = ''
Solution_Steps = []

In [107]:
def ResetParams():
  #Strategy sheet
  global FINAL_TALK_TIME
  global FINAL_CONNECTED_CLIENT
  global FINAL_ATTEMPTED_CLIENT
  global FINAL_SIGNED
  global FINAL_SALES
  global FINAL_LTV
  global FINAL_TELCO_COST
  global FINAL_BONUS
  global FINAL_CALL_COVERAGE
  global FINAL_OPERATION_COST

  #Calculate sheets
  global TOTAL_FTE_SC
  global FTE_Selected
  global FTE_Selected_Remaining
  global FINAL_CLIENT

  #main
  global run
  global errorMsg
  global Solution_Steps

  #Strategy sheet
  FINAL_TALK_TIME = []
  FINAL_CONNECTED_CLIENT = []
  FINAL_ATTEMPTED_CLIENT = []
  FINAL_SIGNED = []
  FINAL_SALES = []
  FINAL_LTV = []
  FINAL_TELCO_COST = []
  FINAL_BONUS = []
  FINAL_CALL_COVERAGE = []
  FINAL_OPERATION_COST = []

  #Calculate sheets
  TOTAL_FTE_SC = []
  TOTAL_FTE_MAX_PERF = []
  TOTAL_FTE_MAX_PERF.append(-1)
  FTE_Selected = 0
  FTE_Selected_Remaining = 0
  FINAL_CLIENT = []

  #main
  run = 1
  errorMsg = ''

  #log
  print('Step 1: Reset parameters')
  print('')
  Solution_Steps = []
  Solution_Steps.append('Step 1: Reset parameters')
  Solution_Steps.append('')

  # delete old data
  for row in MAX_PROFIT_Sheet['A1:AZ1000']:
    for cell in row:
      cell.value = None
  for row in MAX_COVERAGE_Sheet['A1:AZ1000']:
    for cell in row:
      cell.value = None
  for row in MIN_COST_Sheet['A1:AZ1000']:
    for cell in row:
      cell.value = None
  for row in FTE_Sheet['A1:AZ1000']:
    for cell in row:
      cell.value = None
  for row in STRATEGY_Sheet['B2:AZ1000']:
    for cell in row:
      cell.value = None

  return run

##Calculate FTE

In [93]:
import datetime
import calendar

def first_day_of_month(date):
    first_day = datetime.datetime(date.year, date.month, 1)
    return first_day.strftime('%Y-%m-%d')

#print("\nFirst Day of Month: ", first_day_of_month(datetime.date(2020, 2, 25)))

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return (next_month - datetime.timedelta(days=next_month.day)).strftime('%Y-%m-%d')
#print("\nLast Day of Month: ", last_day_of_month(datetime.date(2020, 3, 1)))

def count_Sunday(any_day):
  m = any_day.month
  y = any_day.year
  return sum(1 for week in calendar.monthcalendar(y,m) if week[-1])
#print("\n#Sunday: ", count_Sunday(datetime.date(2020, 11, 1)))

def sortSecond(val): 
  return val[1]


In [94]:
def Calculate_Max_FTE():
  global Solution_Steps
  global run

  #log
  print('Step 2: Calculate days off, working days, FTE monthly in the selected month')
  print('')
  Solution_Steps.append('Step 2: Calculate days off, working days, FTE monthly in the selected month')
  Solution_Steps.append('')

  # delete old data
  for row in FTE_Sheet['A1:AZ1000']:
    for cell in row:
      cell.value = None
  
  # calulate number of days in a month, Sundays in a month, convert headcount to FTE
  FTE_Headers = ["","YEAR","MONTH","START_DATE","END_DATE","CNT_SUNDAY","CNT_WORKING_DAYS","AVG_AGENT_DAILY","MONTH_FTE"]
  FTE_DR = []
  for x in range(1, 14):#row
    try:
      data_cols = []
      for y in range(1, 9):#column
        if x==1:#header
          data_cols.append(FTE_Headers[y])
        else:
          if y==1:#YEAR
            data_cols.append(Calculated_Year)
          elif y==2:#MONTH
            data_cols.append(x-1)
          elif y==3:#START_DATE
            data_cols.append(first_day_of_month(datetime.date(int(Calculated_Year), int(x-1), 1)))
          elif y==4:#END_DATE
            data_cols.append(last_day_of_month(datetime.date(int(Calculated_Year), int(x-1), 1)))
          elif y==5:#CNT_SUNDAY
            sunday = count_Sunday(datetime.date(int(Calculated_Year), int(x-1), 1))
            data_cols.append(sunday)
          elif y==6:#CNT_WORKING_DAYS
            working_days = calendar.monthrange(int(Calculated_Year), int(x-1))[1] - sunday
            data_cols.append(working_days)
          elif y==7:#AVG_AGENT_DAILY
            avg_agent = CallCenter_Input_Sheet.cell(row=2, column=3).value*int(CallCenter_Input_Sheet.cell(row=6, column=3).value[0:1])/int(CallCenter_Input_Sheet.cell(row=6, column=3).value[2:3])
            data_cols.append(avg_agent)
          elif y==8:#MONTH_FTE
            fte = avg_agent*working_days + sunday*CallCenter_Input_Sheet.cell(row=7, column=3).value
            data_cols.append(fte)

            #log
            print('   Rule ER_ID = 26: FINAL_FTE = #working days*avg_agent + #Sundays*fix_agent')
            print('     Working_days = ' + str(working_days) + ', avg_agent = ' + str(avg_agent) + ', sunday = ' + str(sunday) + ', fix_agent = ' + str(CallCenter_Input_Sheet.cell(row=7, column=3).value))
            print('     => FINAL_FTE = ' + str(fte))   
            print('')   
            Solution_Steps.append('   Rule ER_ID = 26: FINAL_FTE = #working days*avg_agent + #Sundays*fix_agent')
            Solution_Steps.append('     Working_days = ' + str(working_days) + ', avg_agent = ' + str(avg_agent) + ', sunday = ' + str(sunday) + ', fix_agent = ' + str(CallCenter_Input_Sheet.cell(row=7, column=3).value))
            Solution_Steps.append('     => FINAL_FTE = ' + str(fte))
            Solution_Steps.append('')

      FTE_DR.append(data_cols)
    except:
      errorMsg = 'Cannot calculate FTE with the provided year'
      Solution_Steps.append(' Error - ' + errorMsg)
      print(' Error - ' + errorMsg)
      run = 0
      break

  # append data row
  i_row=1
  i_col=1
  for row in FTE_DR:
    for cell in row:
      FTE_Sheet.cell(row=i_row, column=i_col).value = cell
      #FTE_Sheet.cell(column=i_col, row=i_row).value = cell
      i_col+=1
    i_row+=1
    i_col=1

  #save
  wb.save('KEA_CH1702008_Assignment.xlsx')
  return run

In [95]:
def Extract_FTE():
  global Solution_Steps
  global run
  #log  
  print('Step 3: Extract FTE from the selected month')
  print('')
  Solution_Steps.append('Step 3: Extract FTE from the selected month')
  Solution_Steps.append('')

  #Extract FTE in selected month
  global FTE_Selected
  global FTE_Selected_Remaining
  #FTE_Selected = 0
  #FTE_Selected_Remaining = 0
  for x in range(1, 14):#row
    try:
      if str(FTE_Sheet.cell(row=x, column=2).value) == str(Calculated_Month):
        FTE_Selected+=FTE_Sheet.cell(row=x, column=8).value

        #log
        print('   Rule RR_ID = 1: If INPUT_MONTH = i Then FINAL_FTE = MONTH_FTE[i]')
        print('     INPUT_MONTH = ' + str(Calculated_Month))
        print('     =>FINAL_FTE = ' + str(FTE_Selected))
        print('')
        Solution_Steps.append('   Rule RR_ID = 1: If INPUT_MONTH = i Then FINAL_FTE = MONTH_FTE[i]')
        Solution_Steps.append('     INPUT_MONTH = ' + str(Calculated_Month))
        Solution_Steps.append('     =>FINAL_FTE = ' + str(FTE_Selected))    
        Solution_Steps.append('')      
    except:
      errorMsg = 'Extract_FTE'
      Solution_Steps.append('Step 3: Error - ' + errorMsg)
      run = 0
      break

  if FTE_Selected==0:
    errorMsg = 'Cannot calculate FTE with provided month'
    print('Issue: Extract_FTE - ' + errorMsg)
    Solution_Steps.append('Issue: Extract_FTE - ' + errorMsg)
    run = 0

  FTE_Selected_Remaining = FTE_Selected
  return run

##Calculate max sales & operation performance

In [96]:
def Calculate_Max_Sales():
  global FTE_Selected
  global FTE_Selected_Remaining
  global Solution_Steps
  global run
  #print(run)
  try:
    #log
    print('Step 4: Extract configuration values for strategy option & calculate segment orders')
    print('')
    Solution_Steps.append('Step 4: Extract configuration values for strategy option & calculate segment orders')
    Solution_Steps.append('')
    
    # delete old data
    for row in MAX_PROFIT_Sheet['A1:AZ1000']:
      for cell in row:
        cell.value = None
    for row in MAX_COVERAGE_Sheet['A1:AZ1000']:
      for cell in row:
        cell.value = None
    for row in MIN_COST_Sheet['A1:AZ1000']:
      for cell in row:
        cell.value = None

    #Extract configuration values for strategy option
    MAX_PROFIT_TKR_DR = []
    MAX_COVERAGE_CD_DR = []
    MIN_COST_TB_DR = []

    #Max Profit Configuration
    for x in range(2, 9):
      Arr_TAKE_UP_RATE = []
      Arr_TAKE_UP_RATE.append(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value)
      for y in range(1, 10):
        if y==Col_TAKE_UP_RATE:
          Arr_TAKE_UP_RATE.append(Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value)
      MAX_PROFIT_TKR_DR.append(Arr_TAKE_UP_RATE)

    MAX_PROFIT_TKR_DR.sort(key = sortSecond, reverse = True)

    MAX_PROFIT_TKR_DR_FINAL = []
    for i in MAX_PROFIT_TKR_DR:
      aa = []
      aa.append(i[0])
      for j in i:
        aa.append(MAX_PROFIT_TKR_DR.index(i) + 1)
        break
      MAX_PROFIT_TKR_DR_FINAL.append(aa)


    #Max Coverage Configuration
    for x in range(2, 9):
      Arr_CALL_DURATION = []
      Arr_CALL_DURATION.append(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value)
      for y in range(1, 10):
        if y==Col_AVG_CALL_DURATION:
          Arr_CALL_DURATION.append(Client_Input_Sheet.cell(row=x, column=Col_AVG_CALL_DURATION).value)
      MAX_COVERAGE_CD_DR.append(Arr_CALL_DURATION)

    MAX_COVERAGE_CD_DR.sort(key = sortSecond)

    MAX_COVERAGE_CD_DR_FINAL = []
    for i in MAX_COVERAGE_CD_DR:
      aa = []
      aa.append(i[0])
      for j in i:
        aa.append(MAX_COVERAGE_CD_DR.index(i) + 1)
        break
      MAX_COVERAGE_CD_DR_FINAL.append(aa)

    #Min Cost Configuration
    for x in range(2, 9):
      Arr_TOTAL_BONUS = []
      Arr_TOTAL_BONUS.append(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value)
      for y in range(1, 10):
        if y==Col_TAKE_UP_RATE:
          Arr_TOTAL_BONUS.append(Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value
                              *Client_Input_Sheet.cell(row=x, column=Col_AVG_BONUS).value
        )
      MIN_COST_TB_DR.append(Arr_TOTAL_BONUS)

    MIN_COST_TB_DR.sort(key = sortSecond)

    MIN_COST_TB_DR_FINAL = []
    for i in MIN_COST_TB_DR:
      aa = []
      aa.append(i[0])
      for j in i:
        aa.append(MIN_COST_TB_DR.index(i) + 1)
        break
      MIN_COST_TB_DR_FINAL.append(aa)


    b=2#copy from Call_Center
    #copy from configuration sheets
    SEGMENT_DR = []
    for x in range(1, 9):
      data_cols = []
      c=2#copy from Call_Center

      #log
      if x>1:#ignore headers
        print(' Calculate for Segment Code = ' + str(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value))
        print('')
        Solution_Steps.append(' Calculate for Segment Code = ' + str(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value))
        Solution_Steps.append('')
      
      for y in range(1, 36):
        if x==1:#header
          #header from Client_Input_Sheet
          if y<10:
            #print(' :', x)
            #print(' :', data_cols)
            data_cols.append(Client_Input_Sheet.cell(row=x, column=y).value)
          elif y < 19: #header from CallCenter_Input_Sheet
            #print(' :', x)
            #print(' :', data_cols)
            data_cols.append(CallCenter_Input_Sheet.cell(row=b, column=2).value)
            b+=1
          else:#configuration data
            #print(' :', x)
            #print(' :', data_cols)
            if y==19:#TOTAL_CONNECTED_CLIENT
              data_cols.append("TOTAL_CONNECTED_CLIENT")
            elif y==20:#TOTAL_CALL_DURATION
              data_cols.append("TOTAL_CALL_DURATION")
            elif y==21:#TOTAL_FTE
              data_cols.append("TOTAL_FTE")
            elif y==22:#TOTAL_SIGNED
              data_cols.append("TOTAL_SIGNED")
            elif y==23:#TOTAL_BONUS
              data_cols.append("TOTAL_BONUS")
            elif y==24:#SEGMENT_SORT
              data_cols.append("SEGMENT_SORT")
            elif y==25:#REQUIRED_FTE
              data_cols.append("REQUIRED_FTE")
            elif y==26:#REQUIRED_TALK_TIME
              data_cols.append("REQUIRED_TALK_TIME")
            elif y==27:#REQUIRED_CONNECTED_CLIENT
              data_cols.append("REQUIRED_CONNECTED_CLIENT")
            elif y==28:#REQUIRED_ATTEMPTED_CLIENT
              data_cols.append("REQUIRED_ATTEMPTED_CLIENT")
            elif y==29:#REQUIRED_SIGNED
              data_cols.append("REQUIRED_SIGNED")
            elif y==30:#REQUIRED_SALES
              data_cols.append("REQUIRED_SALES")
            elif y==31:#REQUIRED_LTV
              data_cols.append("REQUIRED_LTV")
            elif y==32:#REQUIRED_TELCO_COST
              data_cols.append("REQUIRED_TELCO_COST")
            elif y==33:#REQUIRED_BONUS
              data_cols.append("REQUIRED_BONUS")
            elif y==34:#REQUIRED_%_CALL_COVERAGE
              data_cols.append("REQUIRED_%_CALL_COVERAGE")
            elif y==35:#REQUIRED_OPERATION_COST
              data_cols.append("REQUIRED_OPERATION_COST")
        else:
          if y<10:
            if y==4:#total clients
              FINAL_CLIENT.append(Client_Input_Sheet.cell(row=x, column=y).value)
            data_cols.append(Client_Input_Sheet.cell(row=x, column=y).value)
          elif y<19:
            data_cols.append(CallCenter_Input_Sheet.cell(row=c, column=3).value)
            c+=1
          else:
            if y==19:#TOTAL_CONNECTED_CLIENT
              v19 = Client_Input_Sheet.cell(row=x, column=Col_CNT_CLIENT).value * Client_Input_Sheet.cell(row=x, column=Col_CONNECTION_RATE).value
              data_cols.append(v19)

              #log
              print('   Rule ER_ID = 1: TOTAL_CONNECTED_CLIENT = CNT_CLIENT*CONNECTION_RATE')
              print('     CNT_CLIENT = ' + str(Client_Input_Sheet.cell(row=x, column=Col_CNT_CLIENT).value) + ', CONNECTION_RATE = ' + str(Client_Input_Sheet.cell(row=x, column=Col_CONNECTION_RATE).value))
              print('     =>TOTAL_CONNECTED_CLIENT = ' + str(v19))
              print('')
              Solution_Steps.append('   Rule ER_ID = 1: TOTAL_CONNECTED_CLIENT = CNT_CLIENT*CONNECTION_RATE')
              Solution_Steps.append('     CNT_CLIENT = ' + str(Client_Input_Sheet.cell(row=x, column=Col_CNT_CLIENT).value) + ', CONNECTION_RATE = ' + str(Client_Input_Sheet.cell(row=x, column=Col_CONNECTION_RATE).value))
              Solution_Steps.append('     =>TOTAL_CONNECTED_CLIENT = ' + str(v19))
              Solution_Steps.append('')

            elif y==20:#TOTAL_CALL_DURATION
              v20 = Client_Input_Sheet.cell(row=x, column=Col_AVG_CALL_DURATION).value * v19
              data_cols.append(v20)

              #log
              print('   Rule ER_ID = 2: TOTAL_CALL_DURATION = AVG_CALL_DURATION*TOTAL_CONNECTED_CLIENT')
              print('     AVG_CALL_DURATION = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVG_CALL_DURATION).value) + ', TOTAL_CONNECTED_CLIENT = ' + str(v19))
              print('     =>TOTAL_CALL_DURATION = ' + str(v20))
              print('')
              Solution_Steps.append('   Rule ER_ID = 2: TOTAL_CALL_DURATION = AVG_CALL_DURATION*TOTAL_CONNECTED_CLIENT ')
              Solution_Steps.append('     AVG_CALL_DURATION = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVG_CALL_DURATION).value) + ', TOTAL_CONNECTED_CLIENT = ' + str(v19))
              Solution_Steps.append('     =>TOTAL_CALL_DURATION = ' + str(v20))
              Solution_Steps.append('')

            elif y==21:#TOTAL_FTE
              v21 = v20/3600/CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value
              data_cols.append(v21)

              #log
              print('   Rule ER_ID = 3: TOTAL_FTE = TOTAL_CALL_DURATION/3600/AVG_TALKTIME')
              print('     TOTAL_CALL_DURATION = ' + str(v20) + ', AVG_TALKTIME = ' + str(CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value))
              print('     =>TOTAL_FTE = ' + str(v21))
              print('')
              Solution_Steps.append('   Rule ER_ID = 3: TOTAL_FTE = TOTAL_CALL_DURATION/3600/AVG_TALKTIME')
              Solution_Steps.append('     TOTAL_CALL_DURATION = ' + str(v20) + ', AVG_TALKTIME = ' + str(CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value))
              Solution_Steps.append('     =>TOTAL_FTE = ' + str(v21))
              Solution_Steps.append('')

              #store Segment code & needed FTE
              for i in (1,2):
                aa = []
                aa.append(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value)
                for j in (1,2):
                  aa.append(v21)
                  break
                TOTAL_FTE_SC.append(aa)
                break
              
            elif y==22:#TOTAL_SIGNED
              v22 = v19 * Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value
              data_cols.append(v22)

              #log
              print('   Rule ER_ID = 4: TOTAL_SIGNED = TAKE_UP_RATE*TOTAL_CONNECTED_CLIENT')
              print('     TAKE_UP_RATE = ' + str(Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value) + ', TOTAL_CONNECTED_CLIENT = ' + str(v19))
              print('     =>TOTAL_SIGNED = ' + str(v22))
              print('')
              Solution_Steps.append('   Rule ER_ID = 4: TOTAL_SIGNED = TAKE_UP_RATE*TOTAL_CONNECTED_CLIENT')
              Solution_Steps.append('     TAKE_UP_RATE = ' + str(Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value) + ', TOTAL_CONNECTED_CLIENT = ' + str(v19))
              Solution_Steps.append('     =>TOTAL_SIGNED = ' + str(v22))
              Solution_Steps.append('')

            elif y==23:#TOTAL_BONUS
              v23 = v22 * Client_Input_Sheet.cell(row=x, column=Col_AVG_BONUS).value
              data_cols.append(v23)

              #log
              print('   Rule ER_ID = 5: TOTAL_BONUS = TOTAL_SIGNED*AVG_BONUS')
              print('     TOTAL_SIGNED = ' + str(v22) + ', AVG_BONUS = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVG_BONUS).value))
              print('     =>TOTAL_BONUS = ' + str(v23))
              print('')
              Solution_Steps.append('   Rule ER_ID = 5: TOTAL_BONUS = TOTAL_SIGNED*AVG_BONUS')
              Solution_Steps.append('     TOTAL_SIGNED = ' + str(v22) + ', AVG_BONUS = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVG_BONUS).value))
              Solution_Steps.append('     =>TOTAL_BONUS = ' + str(v23))
              Solution_Steps.append('')

            elif y==24:#SEGMENT_SORT
              if Calculated_Option == 'Max Profit':
                for i in MAX_PROFIT_TKR_DR_FINAL:
                  if i[0]==Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value:
                    v24 = i[1]

                    #log
                    print('   Rule RR_ID = 4: If STRATEGY = "MAX PROFIT" Then SEGMENT_SORT = max(TAKE_UP_RATE)')
                    print('     STRATEGY = ' + str(Calculated_Option))
                    print('     =>Priority of Col_SEGMENT_CODE = ' + str(i[1]))
                    print('')
                    Solution_Steps.append('   Rule RR_ID = 4: If STRATEGY = "MAX PROFIT" Then SEGMENT_SORT = max(TAKE_UP_RATE)')
                    Solution_Steps.append('     STRATEGY = ' + str(Calculated_Option))
                    Solution_Steps.append('     =>Priority of Col_SEGMENT_CODE = ' + str(i[1]))
                    Solution_Steps.append('')

                data_cols.append(v24)
              elif Calculated_Option == 'Max Coverage':
                for i in MAX_COVERAGE_CD_DR_FINAL:
                  if i[0]==Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value:
                    v24 = i[1]

                    #log
                    print('   Rule RR_ID = 5: If STRATEGY = "MAX COVERAGE" Then SEGMENT_SORT[1] = min(AVG_CALL_DURATION)')
                    print('     STRATEGY = ' + str(Calculated_Option))
                    print('     =>Priority of Col_SEGMENT_CODE = ' + str(i[1]))
                    print('')
                    Solution_Steps.append('   Rule RR_ID = 4: If STRATEGY = "MAX PROFIT" Then SEGMENT_SORT = max(TAKE_UP_RATE)')
                    Solution_Steps.append('     STRATEGY = ' + str(Calculated_Option))
                    Solution_Steps.append('     =>Priority of Col_SEGMENT_CODE = ' + str(i[1]))
                    Solution_Steps.append('')

                data_cols.append(v24)
              elif Calculated_Option == 'Min Cost':
                for i in MIN_COST_TB_DR_FINAL:
                  if i[0]==Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value:
                    v24 = i[1]

                    #log
                    print('   Rule RR_ID = 6: If STRATEGY = "MIN COST" Then SEGMENT_SORT[1] = min(TOTAL_BONUS)')
                    print('     STRATEGY = ' + str(Calculated_Option))
                    print('     =>Priority of Col_SEGMENT_CODE = ' + str(i[1]))
                    print('')
                    Solution_Steps.append('   Rule RR_ID = 4: If STRATEGY = "MAX PROFIT" Then SEGMENT_SORT = max(TAKE_UP_RATE)')
                    Solution_Steps.append('     STRATEGY = ' + str(Calculated_Option))
                    Solution_Steps.append('     =>Priority of Col_SEGMENT_CODE = ' + str(i[1]))
                    Solution_Steps.append('')
                data_cols.append(v24)
              else:
                errorMsg = 'Cannot sort segment codes based on the provided Strategy Option'
                Solution_Steps.append(' Error - ' + errorMsg)
                print(' Error - ' + errorMsg)
                run = 0
                break

            elif y==25:#REQUIRED_FTE
              v25 = 0
              data_cols.append(v25)
            elif y==26:#REQUIRED_TALK_TIME
              v26 = v25*3600*CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value
              data_cols.append(v26)
            elif y==27:#REQUIRED_CONNECTED_CLIENT
              v27 = v26/Client_Input_Sheet.cell(row=x, column=Col_AVG_CALL_DURATION).value
              data_cols.append(v27)
            elif y==28:#REQUIRED_ATTEMPTED_CLIENT
              v28 = v27/Client_Input_Sheet.cell(row=x, column=Col_CONNECTION_RATE).value
              data_cols.append(v28)
            elif y==29:#REQUIRED_SIGNED
              v29 = v27*Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value
              data_cols.append(v29)
            elif y==30:#REQUIRED_SALES
              v30 = v29*Client_Input_Sheet.cell(row=x, column=Col_AVERAGE_TICKET).value
              data_cols.append(v30)
            elif y==31:#REQUIRED_LTV
              v31 = v29*CallCenter_Input_Sheet.cell(row=Row_LTV, column=3).value
              data_cols.append(v31)
            elif y==32:#REQUIRED_TELCO_COST
              v32 = v26/60*CallCenter_Input_Sheet.cell(row=Row_TELCO_COST, column=3).value
              data_cols.append(v32)
            elif y==33:#REQUIRED_BONUS
              v33 = v29*Client_Input_Sheet.cell(row=x, column=Col_AVG_BONUS).value
              data_cols.append(v33)
            elif y==34:#REQUIRED_%_CALL_COVERAGE
              v34 = v28/Client_Input_Sheet.cell(row=x, column=Col_CNT_CLIENT).value
              data_cols.append(v34)
            elif y==35:#REQUIRED_OPERATION_COST
              v35 = v32 + v33
              data_cols.append(v35)
      #log
      if x>1:#ignore headers
        print(' End ' + str(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value))
        print('')
        Solution_Steps.append(' End ' + str(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value))
        Solution_Steps.append('')
      
      SEGMENT_DR.append(data_cols)

    # append data row
    i_row=1
    i_col=1
    for row in SEGMENT_DR:
      for cell in row:
        if Calculated_Option == 'Max Profit':
          MAX_PROFIT_Sheet.cell(row=i_row, column=i_col).value = cell
        elif Calculated_Option == 'Max Coverage':
          MAX_COVERAGE_Sheet.cell(row=i_row, column=i_col).value = cell
        elif Calculated_Option == 'Min Cost':
          MIN_COST_Sheet.cell(row=i_row, column=i_col).value = cell
        i_col+=1
      i_row+=1
      i_col=1

    #order of segment to be executed
    #"Max Profit", "Max Coverage", "Min Cost", "All"
    if Calculated_Option == 'Max Profit':
      for i in MAX_PROFIT_TKR_DR_FINAL:
        for j in TOTAL_FTE_SC:
          if i[0]==j[0] and FTE_Selected_Remaining >= j[1]:
            MAX_PROFIT_Sheet.cell(row=TOTAL_FTE_SC.index(j)+2, column=25).value = j[1]
            FTE_Selected_Remaining-=j[1]
          elif i[0]==j[0] and FTE_Selected_Remaining < j[1] and FTE_Selected_Remaining > 0:
            MAX_PROFIT_Sheet.cell(row=TOTAL_FTE_SC.index(j)+2, column=25).value = FTE_Selected_Remaining
            FTE_Selected_Remaining=0
    elif Calculated_Option == 'Max Coverage':
      for i in MAX_COVERAGE_CD_DR_FINAL:
        for j in TOTAL_FTE_SC:
          if i[0]==j[0] and FTE_Selected_Remaining >= j[1]:
            MAX_COVERAGE_Sheet.cell(row=TOTAL_FTE_SC.index(j)+2, column=25).value = j[1]
            FTE_Selected_Remaining-=j[1]
          elif i[0]==j[0] and FTE_Selected_Remaining < j[1] and FTE_Selected_Remaining > 0:
            MAX_COVERAGE_Sheet.cell(row=TOTAL_FTE_SC.index(j)+2, column=25).value = FTE_Selected_Remaining
            FTE_Selected_Remaining=0
    elif Calculated_Option == 'Min Cost':
      for i in MIN_COST_TB_DR_FINAL:
        for j in TOTAL_FTE_SC:
          if i[0]==j[0] and FTE_Selected_Remaining >= j[1]:
            MIN_COST_Sheet.cell(row=TOTAL_FTE_SC.index(j)+2, column=25).value = j[1]
            FTE_Selected_Remaining-=j[1]
          elif i[0]==j[0] and FTE_Selected_Remaining < j[1] and FTE_Selected_Remaining > 0:
            MIN_COST_Sheet.cell(row=TOTAL_FTE_SC.index(j)+2, column=25).value = FTE_Selected_Remaining
            FTE_Selected_Remaining=0
    else:
      errorMsg = 'Cannot calculate Required FTE based on the provided Strategy Option'
      Solution_Steps.append(' Error - ' + errorMsg)
      print(' Error - ' + errorMsg)
      run = 0

    #save
    wb.save('KEA_CH1702008_Assignment.xlsx')
  
  except:
    errorMsg = 'Extract configuration values for strategy option & calculate segment orders'
    Solution_Steps.append('Step 4: Error - ' + errorMsg)
    print('Step 4: Error - ' + errorMsg)
    run = 0
  return run

##Calculate required headcount & related cost, sales

In [97]:
def Calculate_Operation_Activities():
  global Solution_Steps
  global run
  try:
    #log
    print('')
    print('Step 5: Calculate required headcount & related cost, sales')
    Solution_Steps.append('Step 5: Calculate required headcount & related cost, sales')
    Solution_Steps.append('')

    #calculate
    SEGMENT_CALCULATE_DR = []
    for x in range(2, 9):
      Cal_data_cols = []

      #log
      print(' Calculate for Segment Code = ' + str(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value))
      print('')
      Solution_Steps.append(' Calculate for Segment Code = ' + str(Client_Input_Sheet.cell(row=x, column=Col_SEGMENT_CODE).value))
      Solution_Steps.append('')

      for y in range(26, 36):
        if y==26:#REQUIRED_TALK_TIME
          v26_REQUIRED_FTE = 0
          if Calculated_Option == 'Max Profit':
            v26_REQUIRED_FTE = MAX_PROFIT_Sheet.cell(row=x, column=25).value
            v26 = MAX_PROFIT_Sheet.cell(row=x, column=25).value*3600*CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value
          elif Calculated_Option == 'Max Coverage':
            v26_REQUIRED_FTE = MAX_COVERAGE_Sheet.cell(row=x, column=25).value
            v26 = MAX_COVERAGE_Sheet.cell(row=x, column=25).value*3600*CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value
          elif Calculated_Option == 'Min Cost':
            v26_REQUIRED_FTE = MIN_COST_Sheet.cell(row=x, column=25).value
            v26 = MIN_COST_Sheet.cell(row=x, column=25).value*3600*CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value
          else:
            errorMsg = 'Cannot calculate Required Talk time based on the provided Strategy Option'
            Solution_Steps.append(' Error - ' + errorMsg)
            print(' Error - ' + errorMsg)
            run = 0
          
          Cal_data_cols.append(v26)
          FINAL_TALK_TIME.append(v26)

          #log
          print('   Rule ER_ID = 6: REQUIRED_TALK_TIME = REQUIRED_FTE*AVG_TALKTIME*3600')
          print('     REQUIRED_FTE = ' + str(v26_REQUIRED_FTE) + ', AVG_TALKTIME = ' + str(CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value))
          print('     =>REQUIRED_TALK_TIME = ' + str(v26))
          print('')
          Solution_Steps.append('   Rule ER_ID = 6: REQUIRED_TALK_TIME = REQUIRED_FTE*AVG_TALKTIME*3600')
          Solution_Steps.append('     REQUIRED_FTE = ' + str(v26_REQUIRED_FTE) + ', AVG_TALKTIME = ' + str(CallCenter_Input_Sheet.cell(row=Row_AVG_TALKTIME, column=3).value))
          Solution_Steps.append('     =>REQUIRED_TALK_TIME = ' + str(v26))
          Solution_Steps.append('')
          
        elif y==27:#REQUIRED_CONNECTED_CLIENT
          v27 = v26/Client_Input_Sheet.cell(row=x, column=Col_AVG_CALL_DURATION).value
          Cal_data_cols.append(v27)
          FINAL_CONNECTED_CLIENT.append(v27)

          #log
          print('   Rule ER_ID = 7: REQUIRED_CONNECTED_CLIENT = REQUIRED_TALK_TIME/ AVG_CALL_DURATION')
          print('     REQUIRED_TALK_TIME = ' + str(v26) + ', AVG_CALL_DURATION = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVG_CALL_DURATION).value))
          print('     =>REQUIRED_CONNECTED_CLIENT = ' + str(v27))
          print('')
          Solution_Steps.append('   Rule ER_ID = 7: REQUIRED_CONNECTED_CLIENT = REQUIRED_TALK_TIME/ AVG_CALL_DURATION')
          Solution_Steps.append('     REQUIRED_TALK_TIME = ' + str(v26) + ', AVG_CALL_DURATION = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVG_CALL_DURATION).value))
          Solution_Steps.append('     =>REQUIRED_CONNECTED_CLIENT = ' + str(v27))
          Solution_Steps.append('')

        elif y==28:#REQUIRED_ATTEMPTED_CLIENT
          v28 = v27/Client_Input_Sheet.cell(row=x, column=Col_CONNECTION_RATE).value
          Cal_data_cols.append(v28)
          FINAL_ATTEMPTED_CLIENT.append(v28)

          #log
          print('   Rule ER_ID = 8: REQUIRED_ATTEMPTED_CLIENT = REQUIRED_CONNECTED_CLIENT/ CONNECTION_RATE')
          print('     REQUIRED_CONNECTED_CLIENT = ' + str(v27) + ', CONNECTION_RATE = ' + str(Client_Input_Sheet.cell(row=x, column=Col_CONNECTION_RATE).value))
          print('     =>REQUIRED_ATTEMPTED_CLIENT = ' + str(v28))
          print('')
          Solution_Steps.append('   Rule ER_ID = 8: REQUIRED_ATTEMPTED_CLIENT = REQUIRED_CONNECTED_CLIENT/ CONNECTION_RATE')
          Solution_Steps.append('     REQUIRED_CONNECTED_CLIENT = ' + str(v27) + ', CONNECTION_RATE = ' + str(Client_Input_Sheet.cell(row=x, column=Col_CONNECTION_RATE).value))
          Solution_Steps.append('     =>REQUIRED_ATTEMPTED_CLIENT = ' + str(v28))
          Solution_Steps.append('')

        elif y==29:#REQUIRED_SIGNED
          v29 = v27*Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value
          Cal_data_cols.append(v29)
          FINAL_SIGNED.append(v29)

          #log
          print('   Rule ER_ID = 9: REQUIRED_SIGNED = REQUIRED_CONNECTED_CLIENT*TAKE_UP_RATE')
          print('     REQUIRED_CONNECTED_CLIENT = ' + str(v27) + ', TAKE_UP_RATE = ' + str(Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value))
          print('     =>REQUIRED_SIGNED = ' + str(v29))
          print('')
          Solution_Steps.append('   Rule ER_ID = 9: REQUIRED_SIGNED = REQUIRED_CONNECTED_CLIENT*TAKE_UP_RATE')
          Solution_Steps.append('     REQUIRED_CONNECTED_CLIENT = ' + str(v27) + ', TAKE_UP_RATE = ' + str(Client_Input_Sheet.cell(row=x, column=Col_TAKE_UP_RATE).value))
          Solution_Steps.append('     =>REQUIRED_SIGNED = ' + str(v29))
          Solution_Steps.append('')

        elif y==30:#REQUIRED_SALES
          v30 = v29*Client_Input_Sheet.cell(row=x, column=Col_AVERAGE_TICKET).value
          Cal_data_cols.append(v30)
          FINAL_SALES.append(v30)

          #log
          print('   Rule ER_ID = 10: REQUIRED_SALES = REQUIRED_SIGNED*AVERAGE_TICKET')
          print('     REQUIRED_SIGNED = ' + str(v29) + ', AVERAGE_TICKET = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVERAGE_TICKET).value))
          print('     =>REQUIRED_SALES = ' + str(v30))
          print('')
          Solution_Steps.append('   Rule ER_ID = 10: REQUIRED_SALES = REQUIRED_SIGNED*AVERAGE_TICKET')
          Solution_Steps.append('     REQUIRED_SIGNED = ' + str(v29) + ', AVERAGE_TICKET = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVERAGE_TICKET).value))
          Solution_Steps.append('     =>REQUIRED_SALES = ' + str(v30))
          Solution_Steps.append('')

        elif y==31:#REQUIRED_LTV
          v31 = v29*CallCenter_Input_Sheet.cell(row=Row_LTV, column=3).value
          Cal_data_cols.append(v31)
          FINAL_LTV.append(v31)

          #log
          print('   Rule ER_ID = 11: REQUIRED_LTV = REQUIRED_SIGNED*LTV')
          print('    REQUIRED_SIGNED = ' + str(v29) + ', LTV = ' + str(CallCenter_Input_Sheet.cell(row=Row_LTV, column=3).value))
          print('     =>REQUIRED_LTV = ' + str(v31))
          print('')
          Solution_Steps.append('   Rule ER_ID = 11: REQUIRED_LTV = REQUIRED_SIGNED*LTV')
          Solution_Steps.append('     REQUIRED_SIGNED = ' + str(v29) + ', LTV = ' + str(CallCenter_Input_Sheet.cell(row=Row_LTV, column=3).value))
          Solution_Steps.append('     =>REQUIRED_LTV = ' + str(v31))
          Solution_Steps.append('')

        elif y==32:#REQUIRED_TELCO_COST
          v32 = v26/60*CallCenter_Input_Sheet.cell(row=Row_TELCO_COST, column=3).value
          Cal_data_cols.append(v32)
          FINAL_TELCO_COST.append(v32)

          #log
          print('   Rule ER_ID = 12: REQUIRED_TELCO_COST = REQUIRED_TALK_TIME/60* TELCO_COST')
          print('     REQUIRED_TALK_TIME = ' + str(v26) + ', TELCO_COST = ' + str(CallCenter_Input_Sheet.cell(row=Row_TELCO_COST, column=3).value))
          print('    =>REQUIRED_TELCO_COST = ' + str(v32))
          print('')
          Solution_Steps.append('   Rule ER_ID = 12: REQUIRED_TELCO_COST = REQUIRED_TALK_TIME/60* TELCO_COST')
          Solution_Steps.append('    REQUIRED_TALK_TIME = ' + str(v26) + ', TELCO_COST = ' + str(CallCenter_Input_Sheet.cell(row=Row_TELCO_COST, column=3).value))
          Solution_Steps.append('     =>REQUIRED_TELCO_COST = ' + str(v32))
          Solution_Steps.append('')

        elif y==33:#REQUIRED_BONUS
          v33 = v29*Client_Input_Sheet.cell(row=x, column=Col_AVG_BONUS).value
          Cal_data_cols.append(v33)
          FINAL_BONUS.append(v33)

          #log
          print('   Rule ER_ID = 13: REQUIRED_BONUS = REQUIRED_SIGNED*AVG_BONUS')
          print('     REQUIRED_SIGNED = ' + str(v29) + ', AVG_BONUS = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVG_BONUS).value))
          print('     =>REQUIRED_BONUS = ' + str(v33))
          print('')
          Solution_Steps.append('   Rule ER_ID = 13: REQUIRED_BONUS = REQUIRED_SIGNED*AVG_BONUS')
          Solution_Steps.append('     REQUIRED_SIGNED = ' + str(v29) + ', AVG_BONUS = ' + str(Client_Input_Sheet.cell(row=x, column=Col_AVG_BONUS).value))
          Solution_Steps.append('     =>REQUIRED_BONUS = ' + str(v33))
          Solution_Steps.append('')

        elif y==34:#REQUIRED_%_CALL_COVERAGE
          v34 = v28/Client_Input_Sheet.cell(row=x, column=Col_CNT_CLIENT).value
          Cal_data_cols.append(v34)
          FINAL_CALL_COVERAGE.append(v34)

          #log
          print('   Rule ER_ID = 14: REQUIRED_%_CALL_COVERAGE = REQUIRED_ATTEMPTED_CLIENT/CNT_CLIENT')
          print('     REQUIRED_ATTEMPTED_CLIENT = ' + str(v28) + ', CNT_CLIENT = ' + str(Client_Input_Sheet.cell(row=x, column=Col_CNT_CLIENT).value))
          print('     =>REQUIRED_%_CALL_COVERAGE = ' + str(v34))
          print('')
          Solution_Steps.append('   Rule ER_ID = 14: REQUIRED_%_CALL_COVERAGE = REQUIRED_ATTEMPTED_CLIENT/CNT_CLIENT')
          Solution_Steps.append('    REQUIRED_ATTEMPTED_CLIENT = ' + str(v28) + ', CNT_CLIENT = ' + str(Client_Input_Sheet.cell(row=x, column=Col_CNT_CLIENT).value))
          Solution_Steps.append('     =>REQUIRED_%_CALL_COVERAGE = ' + str(v34))
          Solution_Steps.append('')

        elif y==35:#REQUIRED_OPERATION_COST
          v35 = v32 + v33
          Cal_data_cols.append(v35)
          FINAL_OPERATION_COST.append(v35)

          #log
          print('   Rule ER_ID = 15: REQUIRED_OPERATION_COST = REQUIRED_TELCO_COST+REQUIRED_BONUS')
          print('     REQUIRED_TELCO_COST = ' + str(v32) + ', REQUIRED_BONUS = ' + str(v33))
          print('     =>REQUIRED_OPERATION_COST = ' + str(v35))
          print('')
          Solution_Steps.append('   Rule ER_ID = 15: REQUIRED_OPERATION_COST = REQUIRED_TELCO_COST+REQUIRED_BONUS')
          Solution_Steps.append('    REQUIRED_TELCO_COST = ' + str(v32) + ', REQUIRED_BONUS = ' + str(v33))
          Solution_Steps.append('     =>REQUIRED_OPERATION_COST = ' + str(v35))
          Solution_Steps.append('')

      SEGMENT_CALCULATE_DR.append(Cal_data_cols)


    # append data row
    i_row=2
    i_col=26
    for row in SEGMENT_CALCULATE_DR:
      for cell in row:
        if Calculated_Option == 'Max Profit':
          MAX_PROFIT_Sheet.cell(row=i_row, column=i_col).value = cell
        elif Calculated_Option == 'Max Coverage':
          MAX_COVERAGE_Sheet.cell(row=i_row, column=i_col).value = cell
        elif Calculated_Option == 'Min Cost':
          MIN_COST_Sheet.cell(row=i_row, column=i_col).value = cell
        i_col+=1
      i_row+=1
      i_col=26

    #save
    wb.save('KEA_CH1702008_Assignment.xlsx')

  except:
    errorMsg = 'Calculate required headcount & related cost, sales'
    Solution_Steps.append('Step 5: Error - ' + errorMsg)
    print('Step 5: Error - ' + errorMsg)
    run = 0
  return run

##Strategy

In [98]:
def Propose_Strategy():
  global run
  try:
    #log
    print('Step 6: summarize resutl')
    print('')
    Solution_Steps.append('Step 6: summarize resutl')
    Solution_Steps.append('')

    # delete old data
    for row in STRATEGY_Sheet['B2:AZ1000']:
      for cell in row:
        cell.value = None

    #Calculate
    STRATEGY_DR = []
    for x in range(2, 5):
      strategy_data_cols = []

      #log
      print(' Summarize strategy option = ' + Calculated_Option)
      print('')
      Solution_Steps.append(' Summarize strategy option = ' + Calculated_Option)
      Solution_Steps.append('')

      for y in range(2, 12):
        if y==2:#FINAL_TALK_TIME
          v2 = sum(FINAL_TALK_TIME)
          strategy_data_cols.append(v2)

          #log
          print('   Rule ER_ID = 16: FINAL_TALK_TIME = sum(REQUIRED_TALK_TIME)')
          print('     =>FINAL_TALK_TIME = ' + str(v2))
          print('')
          Solution_Steps.append('   Rule ER_ID = 16: FINAL_TALK_TIME = sum(REQUIRED_TALK_TIME)')
          Solution_Steps.append('     =>FINAL_TALK_TIME = ' + str(v2))
          Solution_Steps.append('')

        elif y==3:#FINAL_CONNECTED_CLIENT
          v3 = sum(FINAL_CONNECTED_CLIENT)
          strategy_data_cols.append(v3)

          #log
          print('   Rule ER_ID = 17: FINAL_CONNECTED_CLIENT = sum(REQUIRED_CONNECTED_CLIENT)')
          print('     =>FINAL_CONNECTED_CLIENT = ' + str(v3))
          print('')
          Solution_Steps.append('   Rule ER_ID = 17: FINAL_CONNECTED_CLIENT = sum(REQUIRED_CONNECTED_CLIENT)')
          Solution_Steps.append('    =>FINAL_CONNECTED_CLIENT = ' + str(v3))
          Solution_Steps.append('')

        elif y==4:#FINAL_ATTEMPTED_CLIENT
          v4 = sum(FINAL_ATTEMPTED_CLIENT)
          strategy_data_cols.append(v4)

          #log
          print('   Rule ER_ID = 18: FINAL_ATTEMPTED_CLIENT = sum(REQUIRED_ATTEMPTED_CLIENT)')
          print('     =>FINAL_ATTEMPTED_CLIENT = ' + str(v4))
          print('')
          Solution_Steps.append('   Rule ER_ID = 18: FINAL_ATTEMPTED_CLIENT = sum(REQUIRED_ATTEMPTED_CLIENT)')
          Solution_Steps.append('     =>FINAL_ATTEMPTED_CLIENT = ' + str(v4))
          Solution_Steps.append('')

        elif y==5:#FINAL_SIGNED
          v5 = sum(FINAL_SIGNED)
          strategy_data_cols.append(v5)

          #log
          print('   Rule ER_ID = 19: FINAL_SIGNED = sum(REQUIRED_SIGNED)')
          print('     =>FINAL_SIGNED = ' + str(v5))
          print('')
          Solution_Steps.append('   Rule ER_ID = 19: FINAL_SIGNED = sum(REQUIRED_SIGNED)')
          Solution_Steps.append('     =>FINAL_SIGNED = ' + str(v5))
          Solution_Steps.append('')

        elif y==6:#FINAL_SALES
          v6 = sum(FINAL_SALES)
          strategy_data_cols.append(v6)
          
          #log
          print('   Rule ER_ID = 20: FINAL_SALES = sum(REQUIRED_SALES)')
          print('     =>FINAL_SALES = ' + str(v6))
          print('')
          Solution_Steps.append('   Rule ER_ID = 20: FINAL_SALES = sum(REQUIRED_SALES)')
          Solution_Steps.append('     =>FINAL_SALES = ' + str(v6))
          Solution_Steps.append('')
          
        elif y==7:#FINAL_LTV
          v7 = sum(FINAL_LTV)
          strategy_data_cols.append(v7)

          #log
          print('   Rule ER_ID = 21: FINAL_LTV = sum(REQUIRED_LTV)')
          print('     =>FINAL_LTV = ' + str(v7))
          print('')
          Solution_Steps.append('   Rule ER_ID = 21: FINAL_LTV = sum(REQUIRED_LTV)')
          Solution_Steps.append('     =>FINAL_LTV = ' + str(v7))
          Solution_Steps.append('')

        elif y==8:#FINAL_TELCO_COST
          v8 = sum(FINAL_TELCO_COST)
          strategy_data_cols.append(v8)

          #log
          print('   Rule ER_ID = 22: FINAL_TELCO_COST = sum(REQUIRED_TELCO_COST)')
          print('     =>FINAL_TELCO_COST = ' + str(v8))
          print('')
          Solution_Steps.append('   Rule ER_ID = 22: FINAL_TELCO_COST = sum(REQUIRED_TELCO_COST)')
          Solution_Steps.append('     =>FINAL_TELCO_COST = ' + str(v8))
          Solution_Steps.append('')

        elif y==9:#FINAL_BONUS
          v9 = sum(FINAL_BONUS)
          strategy_data_cols.append(v9)

          #log
          print('   Rule ER_ID = 23: FINAL_BONUS = sum(REQUIRED_BONUS)')
          print('     =>FINAL_BONUS = ' + str(v9))
          print('')
          Solution_Steps.append('   Rule ER_ID = 23: FINAL_BONUS = sum(REQUIRED_BONUS)')
          Solution_Steps.append('     =>FINAL_BONUS = ' + str(v9))
          Solution_Steps.append('')

        elif y==10:#FINAL_% CALL_COVERAGE
          v10 = sum(FINAL_ATTEMPTED_CLIENT)/sum(FINAL_CLIENT)
          strategy_data_cols.append(v10)

          #log
          print('   Rule ER_ID = 24: FINAL_% CALL_COVERAGE = sum(REQUIRED_ATTEMPTED_CLIENT)/sum(CNT_CLIENT)')
          print('     =>FINAL_% CALL_COVERAGE = ' + str(v10))
          print('')
          Solution_Steps.append('   Rule ER_ID = 24: FINAL_% CALL_COVERAGE = sum(REQUIRED_ATTEMPTED_CLIENT)/sum(CNT_CLIENT)')
          Solution_Steps.append('     =>FINAL_% CALL_COVERAGE = ' + str(v10))
          Solution_Steps.append('')

        elif y==11:#FINAL_OPERATION_COST
          v11 = sum(FINAL_OPERATION_COST)
          strategy_data_cols.append(v11)

          #log
          print('   Rule ER_ID = 25: FINAL_OPERATION_COST = sum(REQUIRED_OPERATION_COST)')
          print('     =>FINAL_OPERATION_COST = ' + str(v11))
          print('')
          Solution_Steps.append('   Rule ER_ID = 25: FINAL_OPERATION_COST = sum(REQUIRED_OPERATION_COST)')
          Solution_Steps.append('     =>FINAL_OPERATION_COST = ' + str(v11))
          Solution_Steps.append('')

      STRATEGY_DR.append(strategy_data_cols)


    # append data row
    i_row=2
    i_col=2
    for row in STRATEGY_DR:
      for cell in row:
        if cell ==  '':
          print('Issue: Propose_Strategy')
          errorMsg = 'Propose_Strategy'
          run = 0
          break;
        elif Calculated_Option == 'Max Profit':
          STRATEGY_Sheet.cell(row=2, column=i_col).value = cell
        elif Calculated_Option == 'Max Coverage':
          STRATEGY_Sheet.cell(row=3, column=i_col).value = cell
        elif Calculated_Option == 'Min Cost':
          STRATEGY_Sheet.cell(row=4, column=i_col).value = cell
        i_col+=1
      i_row+=1
      i_col=2

    #save
    wb.save('KEA_CH1702008_Assignment.xlsx')
  
  except:
    errorMsg = 'Strategy'
    Solution_Steps.append('Step 6: Error - ' + errorMsg)
    print('Step 6: Error - ' + errorMsg)
    run = 0
  return run

##Main

In [110]:
global run

#reset program params
ResetParams()

# delete old data
for row in STEPS_Sheet['A1:AZ1000']:
  for cell in row:
    cell.value = None

#main
while run == 1:
  if Calculate_Max_FTE()==0:
    break
  if Extract_FTE()==0:
    break
  if Calculate_Max_Sales()==0:
    break
  if Calculate_Operation_Activities()==0:
    break
  if Propose_Strategy()==0:
    break
  print('Finished')
  break
else:
  print(run)
  print(errorMsg)
  print('Cannot provide solution based on input parameters')

#Write steps
i = 1
for step in Solution_Steps:
  STEPS_Sheet.cell(row=i, column=1).value = step
  i+=1
#save
wb.save('KEA_CH1702008_Assignment.xlsx')

Step 1: Reset parameters

Step 2: Calculate days off, working days, FTE monthly in the selected month

   Rule ER_ID = 26: FINAL_FTE = #working days*avg_agent + #Sundays*fix_agent
     Working_days = 26, avg_agent = 214.28571428571428, sunday = 5, fix_agent = 80
     => FINAL_FTE = 5971.428571428572

   Rule ER_ID = 26: FINAL_FTE = #working days*avg_agent + #Sundays*fix_agent
     Working_days = 24, avg_agent = 214.28571428571428, sunday = 4, fix_agent = 80
     => FINAL_FTE = 5462.857142857143

   Rule ER_ID = 26: FINAL_FTE = #working days*avg_agent + #Sundays*fix_agent
     Working_days = 27, avg_agent = 214.28571428571428, sunday = 4, fix_agent = 80
     => FINAL_FTE = 6105.714285714285

   Rule ER_ID = 26: FINAL_FTE = #working days*avg_agent + #Sundays*fix_agent
     Working_days = 26, avg_agent = 214.28571428571428, sunday = 4, fix_agent = 80
     => FINAL_FTE = 5891.428571428572

   Rule ER_ID = 26: FINAL_FTE = #working days*avg_agent + #Sundays*fix_agent
     Working_days = 26, 